In [90]:
import numpy as np
import cv2
import tensorflow as tf

In [91]:
face_clsfr = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

model = tf.keras.models.load_model('200-100.model')

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,0,0),1:(255,255,255)}

In [102]:
source = cv2.VideoCapture(0)
 
while(True):
    ret, frame = source.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_clsfr.detectMultiScale(gray,1.3,5)
    
    for (x, y, w, h) in faces:
        face_img = gray[y:y + w, x:x + w]
        resized = cv2.resize(face_img, (120,120))
        normalized = resized / 255.0
        reshaped = np.reshape(normalized, (-1,120,120,1))
        result = model.predict(reshaped)
        label = np.argmax(result,axis=1)[0]
        cv2.rectangle(face_img, (0,0), (0+w, 0+h) ,color_dict[label], 2)
        cv2.putText(face_img, labels_dict[label], (5, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color_dict[label], 1)
        cv2.imshow('frame', gray)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

source.release()
cv2.destroyAllWindows()